# Inheco Incubator (Shaker)

| Summary | Photo |
|----------|--------|
| - [OEM Link](https://www.inheco.com/incubator-shaker.html)<br>- **Communication Protocol / Hardware**: Serial (FTDI)/ USB-A<br>- **Communication Level**: Firmware (documentation shared by OEM)<br>- Same command set for:<ul><li>Incubator "MP"</li><li>Incubator "DWP"</li><li>Incubator Shaker "MP"</li><li>Incubator Shaker "DWP"</li></ul>- Incubator Shaker "MP" VID:PID 0403:6001<br>- Takes in a single plate via a loading tray, heats it to set temperature, shakes it to set RPM | ![quadrants](img/inheco_incubator_shaker_mp_dwp.png) |


This notebook provides a minimal working example to connect to and interact with an INHECO Incubator Shaker over a USB serial connection. It demonstrates how to initialize communication, issue basic commands, and interpret the device’s responses.

## About the Device and Setup

INHECO incubator shakers are modular devices used for temperature control and agitation in lab automation setups. Multiple devices can be connected in a stack via USB, each assigned a **device ID** and **subdevice number** (usually 0–5). Commands are sent via a serial protocol and must be carefully constructed with CRC checks and timeouts.

Ensure:
- The correct **serial port** is selected (e.g., `/dev/cu.usbserial-140` on macOS).
- You know the **device ID** for the incubator you're communicating with.
- Only one command is sent at a time — the device cannot queue or process multiple requests concurrently.

## Setup Instructions (Physical)

| Summary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Photo                                            |
|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------|
| ![quadrants](img/inheco_incubator_shaker_physical_setup_overview.png) | |

## Setup Instructions (Programmatic)

In [1]:
import asyncio
import serial.tools.list_ports
import logging
from typing import Optional, Literal

import logging

from pylabrobot.io.serial import Serial

try:
  import serial

  HAS_SERIAL = True
except ImportError as e:
  HAS_SERIAL = False
  _SERIAL_IMPORT_ERROR = e



class InhecoError(RuntimeError):
    """Represents a firmware-reported INHECO error."""
    def __init__(self, command: str, code: str, message: str):
        super().__init__(f"{command} failed with error {code}: {message}")
        self.command = command
        self.code = code
        self.message = message


_REF_FLAG_NAMES = {
    # Heater (0–15) — names per manual’s heater flags table (subset shown here)
    0:  "H_WARN_WarmUp_TIME",
    1:  "H_WARN_BoostCoolDown_TIME",
    2:  "H_WARN_StartState_LIMIT_Up_TEMP_S2",
    3:  "H_WARN_StartState_LIMIT_Up_TEMP_S3",
    4:  "H_WARN_StartStateBoost_LIMIT_UpDown_TEMP_S3",
    5:  "H_WARN_StableState_LIMIT_UpDown_TEMP_S2",
    6:  "H_WARN_StableState_LIMIT_UpDown_TEMP_S3",
    7:  "H_WARN_DELTA_TEMP_S1_S2",
    8:  "H_ERR_DELTA_TEMP_S1_S2",
    9:  "H_WARN_StartStateBoost_LIMIT_UpDown_TEMP_S2",
    10: "H_WARN_WaitStable_LIMIT_TEMP_S1",
    11: "H_WARN_WaitStable_LIMIT_TEMP_S2",
    12: "H_WARN_WaitStable_LIMIT_TEMP_S3",
    13: "H_ERR_S2_NTC_NotConnected",
    14: "H_ERR_S3_NTC_NotConnected",
    15: "H_WARN_DELTA_TEMP_S1_S3",

    # Shaker (16–26) — names per manual’s shaker flag set (page 39)
    16: "S_WARN_MotorCurrentLimit",
    17: "S_WARN_TargetSpeedTimeout",
    18: "S_WARN_PositionTimeout",
    19: "S_ERR_MotorTemperatureLimit",
    20: "S_ERR_TargetSpeedDeviation",
    21: "S_ERR_HomeSensorTimeout",
    22: "S_ERR_MotorDriverFault",
    23: "S_ERR_EncoderSignalLost",
    24: "S_ERR_AmplitudeOutOfRange",
    25: "S_ERR_VibrationExcessive",
    26: "S_ERR_InternalTimeout",
    # 27–31 reserved
}

FIRMWARE_ERROR_MAP = {
    0: "Msg Ok",
    1: "Reset detected",
    2: "Invalid command",
    3: "Invalid operand",
    4: "Protocol error",
    5: "Reserved",
    6: "Timeout from Device",
    7: "Device not initialized",
    8: "Command not executable",
    9: "Drawer not in end position",
    10: "Unexpected Labware Status",
    13: "Drawer DWP not perfectly closed (NTC not connected)",
    14: "Floor ID error",
    15: "Timeout sub device",
}


class InhecoIncubatorShakerBackend:
    """Async backend for controlling an INHECO Incubator/Shaker via USB VCP."""

        # === Logging helpers ===

    def _log(self, level: int, message: str, direction: Optional[str] = None):
        """
        Unified logging with a clear device tag and optional direction marker.
        direction: "→" for TX, "←" for RX, None for neutral.
        """
        prefix = f"[INHECO IncShak dip_switch={self.dip_switch_id} stack_idx={self.stack_index}]"
        if direction:
            prefix += f" {direction}"
        self.logger.log(level, f"{prefix} {message}")


    def __init__(
        self,
        port: Optional[str] = None,
        dip_switch_id: int = 2,
        stack_index: int = 0,
        write_timeout: float = 5.0,
        read_timeout: float = 10.0,
        logger: Optional[logging.Logger] = None,
    ):
        # Autodetect FTDI if not provided
        VID = "0403"
        PID = "6001"

        if port is None:
            matching_ports = [
                p.device
                for p in serial.tools.list_ports.comports()
                if f"{VID}:{PID}" in (p.hwid or "")
            ]
            if not matching_ports:
                raise RuntimeError(f"No FTDI ({VID}:{PID}) serial devices found.")
            if len(matching_ports) > 1:
                ports_list = ", ".join(matching_ports)
                raise RuntimeError(
                    "Multiple INHECO devices found for VID:PID="
                    f"{VID}:{PID}: {ports_list}. Provide 'port'."
                )
            port = matching_ports[0]

        self.port = port
        self.dip_switch_id = dip_switch_id
        self.stack_index = stack_index
        self.ser: Optional[serial.Serial] = None
        self.write_timeout = write_timeout
        self.read_timeout = read_timeout
        self.logger = logging.getLogger("pylabrobot")

        # --- Logging verbosity control (Option A: clean setup) ---
        # Only show INFO and above for normal runs
        self.logger.setLevel(logging.INFO)
        
        # Silence byte-level I/O logging (the [IO] pylabrobot.io.serial chatter)
        logging.getLogger("pylabrobot.io.serial").disabled = True


        self.setup_finished = False
        self.io = Serial(
            port=self.port,
            baudrate=19200,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=0,  # non-blocking; we manage timeouts in asyncio loop
            write_timeout=self.write_timeout,
        )
        
        # Cached machine status
        self.loading_tray = "unknown"
        self.incubator_type = "unknown"
        self.firmware_version = "unknown"

    # === Lifecycle ===

    async def setup(self, verbose: bool =  False):
        await self.io.setup()  # ✅ open serial port asynchronously
        self.ser = self.io

        self.firmware_version = await self.request_firmware_version()
        incubator_type = await self.request_incubator_type()
        serial_number = await self.request_serial_number()

        msg = (
            f"Connected to INHECO {incubator_type} on {self.port}\n"
            f"Machine serial number: {serial_number}\n"
            f"Firmware version: {self.firmware_version}"
        )

        if verbose:
            print(msg)
        self._log(logging.INFO, msg)

        await self.initialize()
        self.setup_finished = True

    async def stop(self):
        await self.io.stop()  # ✅ proper async cleanup
        self._log(logging.INFO, "Disconnected from INHECO Incubator/Shaker")


    # === Low-level I/O ===

    async def write(self, data: bytes):
        self._log(logging.DEBUG, f"→ {data.hex(' ')}")
        await self.ser.write(data)  # ✅ properly awaited


    async def _read_full_response(self, timeout: float) -> bytes:
        """Read one full INHECO frame asynchronously, combining variable chunks."""
        if not self.ser:
            raise RuntimeError("Serial port not open.")
    
        loop = asyncio.get_event_loop()
        start = loop.time()
        buf = bytearray()
        expected_hdr = (0xB0 + self.dip_switch_id) & 0xFF
    
        def has_complete_tail(b: bytearray) -> bool:
            return (
                len(b) >= 3
                and b[-1] == 0x60
                and b[-3] == expected_hdr
                and 0x20 <= b[-2] <= 0x2F
            )
    
        while True:
            # Try to read up to 16 bytes at once — this limits per-byte log spam
            chunk = await self.ser.read(16)
            if chunk:
                buf.extend(chunk)
                self._log(logging.DEBUG, chunk.hex(" "), direction="←")
    
                if has_complete_tail(buf):
                    return bytes(buf)
    
            # Timeout protection
            if loop.time() - start > timeout:
                raise TimeoutError(
                    f"Timed out waiting for complete response (so far: {buf.hex(' ')})"
                )
    
            # brief pause to yield to event loop, avoid tight spin
            await asyncio.sleep(0.005)

    
    # === Encoding / Decoding ===

    def _crc8_legacy(self, data: bytearray) -> int:
        """
        Legacy CRC used in existing driver (init=0xA1, shift/XOR logic).
        Keep for compatibility with your deployed devices.
        """
        crc = 0xA1
        for byte in data:
            d = byte
            for _ in range(8):
                if (d ^ crc) & 1:
                    crc ^= 0x18
                    crc >>= 1
                    crc |= 0x80
                else:
                    crc >>= 1
                d >>= 1
        return crc & 0xFF

    def _build_message(self, command: str, stack_index: int = 0) -> bytes:
        if not (0 <= stack_index <= 5):
            raise ValueError("stack_index must be between 0 and 5")
        full_command = f"T0{stack_index}{command}"
        cmd = full_command.encode("ascii")
        length = len(cmd) + 3
        address = (0x30 + self.dip_switch_id) & 0xFF
        proto = (0xC0 + len(cmd)) & 0xFF
        message = bytearray([length, address, proto]) + cmd
        crc = self._crc8_legacy(message)
        return bytes(message + bytearray([crc]))

    def _is_report_command(self, command: str) -> bool:
        return command and command[0].upper() == "R"

    # ===== New binary-safe, protocol-accurate response parser =====

    def _parse_response_binary_safe(self, resp: bytes) -> dict:
        """
        Parse INHECO response frames safely (binary & multi-segment).

        Handles:
          - Set/Action:  [B0+ID][20+err][60]
          - Report:      [B0+ID]<data>[B0+ID]... [B0+ID][20+err][60]
          - Also works when only a single [B0+ID] header precedes data.

        Returns:
          dict(
            device=int,
            error_code=int|None,
            ok=bool,
            data=str,
            raw_data=bytes
          )
        """
        if len(resp) < 3:
            raise ValueError("Incomplete response")

        expected_hdr = (0xB0 + self.dip_switch_id) & 0xFF

        # --- Trim leading junk before the first valid header ---
        try:
            start_idx = resp.index(bytes([expected_hdr]))
            frame = resp[start_idx:]
        except ValueError:
            return {
                "device": None,
                "error_code": None,
                "ok": False,
                "data": "",
                "raw_data": resp,
            }

        # --- Validate tail (status section) ---
        if len(frame) < 3 or frame[-1] != 0x60:
            # No valid tail; may be bootloader or incomplete
            return {
                "device": expected_hdr - 0xB0,
                "error_code": None,
                "ok": False,
                "data": "",
                "raw_data": frame,
            }

        # Extract error code (0x20 + err)
        err_byte = frame[-2]
        err_code = err_byte - 0x20 if 0x20 <= err_byte <= 0x2F else None

        # --- Collect data between headers ---
        # Pattern: [hdr] <data> [hdr] <data> ... [hdr] [20+err][60]
        data_blocks = []
        i = 1  # start right after the first header

        while i < len(frame) - 3:
            try:
                next_hdr = frame.index(bytes([expected_hdr]), i)
            except ValueError:
                # No further header — consume until the status tail
                next_hdr = len(frame) - 3

            # Capture bytes between i and next_hdr
            if next_hdr > i:
                data_blocks.append(frame[i:next_hdr])

            i = next_hdr + 1
            if next_hdr >= len(frame) - 3:
                break

        # --- Assemble and decode ---
        raw_data = b"".join(data_blocks)

        try:
            ascii_data = raw_data.decode("ascii").strip("\x00")
        except UnicodeDecodeError:
            ascii_data = raw_data.hex()

        return {
            "device": expected_hdr - 0xB0,
            "error_code": err_code,
            "ok": (err_code == 0),
            "data": ascii_data,
            "raw_data": raw_data,
        }

    def _is_error_tail(self, resp: bytes) -> bool:
        expected_hdr = (0xB0 + self.dip_switch_id) & 0xFF
        return len(resp) >= 3 and resp.endswith(bytes([expected_hdr, 0x28, 0x60]))

    # === High-level Command API ===

    async def send_command(
        self,
        command: str,
        delay: float = 0.2,
        read_timeout: Optional[float] = None,
    ) -> str:
        """
        Send a command and return a string for report commands, or "" otherwise.
        Uses binary-safe, protocol-accurate parsing under the hood.
        """
        msg = self._build_message(command, stack_index=self.stack_index)

        self._log(logging.INFO, f"SENT MESSAGE: {msg}")
        
        await self.write(msg)
        await asyncio.sleep(delay)
        response = await self._read_full_response(
            timeout=read_timeout or self.read_timeout
        )
        if not response:
            raise TimeoutError(f"No response from machine for command: {command}")

        # If firmware returned an error tail, query RER for detail and raise
        if self._is_error_tail(response):
            tail_err = response[-2] - 0x20  # 0..15
            code = f"E{tail_err:02d}"
            message = FIRMWARE_ERROR_MAP.get(tail_err, "Unknown firmware error")
        
            # Optionally gather extra context — never let this block mask the primary error
            ctx = {}
            try:
                ctx = await self._collect_error_context()
                # You can also log this context
                self._log(logging.DEBUG, "Error context: %s", ctx)
            except Exception:
                pass
        
            # Raise with the primary FW error, plus attach context
            err = InhecoError(command, code, message)
            err.context = ctx  # attach structured diagnostics for caller/logs
            raise err

        self._log(logging.INFO, f"RAW RESPONSE: {response}")
        # Parse response frame (works for both report and ack styles)
        parsed = self._parse_response_binary_safe(response)
        self._log(logging.DEBUG, f"PARSED RESPONSE: {parsed}")

        # For report commands, return data; else validate ack-like tail
        if self._is_report_command(command):
            if not parsed["ok"]:
                raise InhecoError(command, "E00", "Report returned non-OK status")
            return parsed["data"]

        # Non-report: require OK
        if not parsed["ok"]:
            code_num = parsed.get("error_code")
            if code_num is not None:
                code = f"E{code_num:02d}"
                message = FIRMWARE_ERROR_MAP.get(code_num, "Unknown firmware error")
            else:
                code = "E00"
                message = "Unknown error (no error code reported)"
        
            # Again, optional context
            ctx = {}
            try:
                ctx = await self._collect_error_context()
                self._log(logging.DEBUG, "Error context: %s", ctx)
            except Exception:
                pass
        
            err = InhecoError(command, code, message)
            err.context = ctx
            raise err

    # === Error Handling ===

        # New debug method (human-readable printout)
    async def debug_error_registry(self):
        """
        Spec-compliant error snapshot using REE/REF/REP.
        - REE: init & labware status (0..3)
        - REF: 32-bit flag mask (bit set = active)
        - REP: heater flag parameters for bits {0..8, 13, 14, 15}
        """
        print("=== ERROR REGISTRY DEBUG ===")
    
        # Use your existing helpers that interpret REE
        try:
            is_init = await self.request_is_initialized()       # uses REE
            plate_known = await self.request_plate_status_known()  # uses REE
            print(f"REE → initialized={is_init}, plate_status_known={plate_known}")
        except Exception as e:
            print(f"REE query failed: {e}")
    
        try:
            ref_raw = await self.send_command("REF")  # returns 32-bit mask as decimal ASCII
            ref_mask = int(ref_raw.strip())
            print(f"REF (flags bitmask): {ref_mask} (0x{ref_mask:08X})")  # 32-bit mask. 
    
            set_bits = [b for b in range(32) if (ref_mask >> b) & 1]
            if not set_bits:
                print(" - No flags set.")
            else:
                print(" - Active flags:")
                for b in set_bits:
                    name = _REF_FLAG_NAMES.get(b, f"Flag{b}")
                    print(f"   [{b:02d}] {name}")
    
                    # REP supports heater selectors {0..8,13,14,15}. 
                    if b in {0,1,2,3,4,5,6,7,8,13,14,15}:
                        try:
                            param = await self.send_command(f"REP{b}")
                            print(f"      → Parameter: {param}")
                        except Exception as e:
                            print(f"      → REP{b} failed: {e}")
    
        except Exception as e:
            print(f"REF/REP read failed: {e}")
    
        print("=== END ERROR REGISTRY DEBUG ===")

    async def _collect_error_context(self) -> dict:
        ctx = {"ree": None, "ref_mask": None, "flags": [], "rep_params": {}}
        try:
            is_init = await self.request_is_initialized()
            plate_known = await self.request_plate_status_known()
            ctx["ree"] = {"initialized": is_init, "plate_status_known": plate_known}
        except Exception:
            pass
    
        try:
            ref_raw = await self.send_command("REF")
            ref_mask = int(ref_raw.strip())
            ctx["ref_mask"] = ref_mask
            set_bits = [b for b in range(32) if (ref_mask >> b) & 1]
            for b in set_bits:
                ctx["flags"].append({"bit": b, "name": _REF_FLAG_NAMES.get(b, f"Flag{b}")})
                if b in {0,1,2,3,4,5,6,7,8,13,14,15}:
                    try:
                        param = await self.send_command(f"REP{b}")
                        ctx["rep_params"][b] = param
                    except Exception:
                        pass
        except Exception:
            pass
    
        return ctx

    # === Public API ===

    async def request_firmware_version(self) -> str:
        # RFV selector: '0' = firmware version, '2' = serial no.
        return await self.send_command("RFV0")

    async def request_serial_number(self) -> str:
        return await self.send_command("RFV2")

    async def request_last_calibration_date(self) -> str:
        resp = await self.send_command("RCM")
        return resp[:10]

    async def request_number_of_connected_machines(self) -> int:
        resp = await self.send_command("RDA")
        return int(resp)

    async def request_labware_detection_threshold(self) -> int:
        resp = await self.send_command("RDM")
        return int(resp)

    async def request_incubator_type(self) -> str:
        # Map per your previous code
        incubator_type_dict = {
            "0": "incubator_mp",         # no shaker
            "1": "incubator_shaker_mp",
            "2": "incubator_dwp",        # no shaker
            "3": "incubator_shaker_dwp",
        }
        resp = await self.send_command("RTS")
        ident = incubator_type_dict.get(resp, "unknown")
        self.incubator_type = ident
        return ident

    async def request_plate_in_incubator(self) -> bool:
        resp = await self.send_command("RLW")
        return resp == "1"

    async def request_operation_time_in_hours(self) -> int:
        resp = await self.send_command("RDC1")
        return int(resp)

    async def request_drawer_cycles_performed(self) -> int:
        resp = await self.send_command("RDC2")
        return int(resp)

    async def request_is_initialized(self) -> bool:
        resp = await self.send_command("REE")
        return resp in {"0", "2"}

    async def request_plate_status_known(self) -> bool:
        resp = await self.send_command("REE")
        return resp in {"0", "1"}

    async def initialize(self):
        return await self.send_command("AID")

    # Drawer

    async def open(self):
        await self.send_command("AOD")
        self.loading_tray = "open"

    async def close(self):
        await self.send_command("ACD")
        self.loading_tray = "closed"

    # Temperature

    async def set_temperature(self, temperature: float, passive: bool = False):
        target = round(temperature * 10)
        await self.send_command(f"STT{target}")
        if not passive:
            await self.send_command("SHE1")

    async def stop_temperature(self):
        await self.send_command("SHE0")

    async def get_temperature(
        self,
        sensor: Literal["mean", "main", "dif", "boost"] = "mean",
    ) -> float:
        sensor_mapping = {
            "mean": [1, 2, 3],
            "main": [1],
            "dif": [2],
            "boost": [3],
        }
        vals = []
        for idx in sensor_mapping[sensor]:
            val = await self.send_command(f"RAT{idx}", read_timeout=60)
            vals.append(int(val) / 10.0)
        return round(sum(vals) / len(vals), 2)

    async def wait_for_temperature(self, timeout: float = 300.0, tolerance: float = 0.5):
        raise NotImplementedError("not yet implemented")

    # Self-Test

    async def perform_self_test(self, read_timeout: int = 500):
        plate_in = await self.request_plate_in_incubator()
        if plate_in:
            raise ValueError("Self-test requires an empty incubator.")
        resp = await self.send_command("AQS", read_timeout=read_timeout)
        return resp




In [2]:
import logging
from pylabrobot.io import LOG_LEVEL_IO
from datetime import datetime
import numpy as np

current_date = datetime.today().strftime('%Y-%m-%d')
protocol_mode = "execution"

# Create the shared file handler once
fh = logging.FileHandler(f"{current_date}_testing_{protocol_mode}.log", mode="a")
fh.setLevel(LOG_LEVEL_IO)
formatter = logging.Formatter(
    "%(asctime)s [%(levelname)s] %(name)s - %(message)s"
)
fh.setFormatter(formatter)

# Configure the main pylabrobot logger
logger_plr = logging.getLogger("pylabrobot")
logger_plr.setLevel(LOG_LEVEL_IO)
if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename
           for h in logger_plr.handlers):
    logger_plr.addHandler(fh)

# Other loggers can reuse the same file handler
logger_manager = logging.getLogger("manager")
logger_device = logging.getLogger("device")

for logger in [logger_manager, logger_device]:
    logger.setLevel(logging.DEBUG)  # or logging.INFO
    if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename
               for h in logger.handlers):
        logger.addHandler(fh)

# START LOGGING
logger_manager.info("START AUTOMATED PROTOCOL")


In [3]:
# Autodetect FTDI device
incubator = InhecoIncubatorShakerBackend(
    dip_switch_id=2,
    stack_index=0
)
await incubator.setup(verbose=True)

Connected to INHECO incubator_shaker_mp on /dev/cu.usbserial-130
Machine serial number: 2013
Firmware version: IncShak_C_V3.50_04/2012


In [4]:
incubator.incubator_type

'incubator_shaker_mp'

In [5]:
await incubator.get_temperature(sensor="main")

36.9

In [6]:
await incubator.set_temperature(37)

In [7]:
import time

await incubator.stop_temperature()

for x in range(20):

    
    resp = await incubator.get_temperature(sensor="main")
    print(resp)

    time.sleep(1)

17.9
18.4
18.5
18.4
18.3
18.3
18.2
18.2
18.1
18.1
18.1
18.1
18.0
18.0
18.0
18.0
18.0
17.9
17.9
17.9


In [13]:
await incubator.perform_self_test()

In [10]:
await incubator.request_plate_in_incubator()

False

In [11]:
await incubator.debug_error_registry()

=== ERROR REGISTRY DEBUG ===
REE → initialized=True, plate_status_known=True
REF (flags bitmask): 0 (0x00000000)
 - No flags set.
=== END ERROR REGISTRY DEBUG ===


## Usage

### Testing & Investigation

In [12]:
await incubator.send_command("REE")

'0'

In [13]:
await incubator.send_command("RDC2")

# Reports the status of the initialisation of the device and the status of the Lab-Ware detection
# if
# 0 Device initialised
# 1 Device not initialised
# 2 Lab-Ware status unknown
# 3 Device not initialised and Lab-Ware status unknow

'1675'

'1'

In [16]:
for x in [
    0, 1, 2, 3, 
    5, 7, 10, 15,
    20, 21, 22, 23,
    24, 25
]:
    print(x, await incubator.send_command(f"RFV{x}") )

# Reports the date and an alphanumeric string (e.g. operator) of the last calibration for the device.
# The Data is reported in the Format YYYY-MM-DD,xxxxx (Example: 2005-09-28,xxxxx).
# The five xs are alphanumeric wildcards.

0 IncShak_C_V3.50_04/2012
1 BOOT_C_V3.12_05/2007
2 2013
3 0
5 COPYRIGHT INHECO
7 1
10 IncShak
15 350
20 325
21 315
22 IncShak_R_V3.25_12/2011
23 BOOT_R_V1.10_01/2007
24 IncShak_S_V3.15_05/2011
25 BOOT_S_V1.10_01/2007


'incubator_shaker_mp'

In [9]:
await incubator.stop()

In [15]:

def request_machine_initialisation_status():
    """ """
    resp = await incubator.send_command("REE")

# Reports the status of the initialisation of the device and the status of the Lab-Ware detection
# if
# 0 Device initialised
# 1 Device not initialised
# 2 Lab-Ware status unknown
# 3 Device not initialised and Lab-Ware status unknow

'0'

### Loading Drawer

In [14]:
await incubator.open_lid()
# await asyncio.sleep(5)
# await incubator.close_lid()

In [16]:
await incubator.close_lid()

### Temperature Control

In [5]:
import time

for t in range(5):
    temp_meas = {}
    
    for x in range(1,4):
        temp_meas[x] = await incubator.send_command(f"RAT{x}", read_timeout=60)

    print(t, temp_meas)
    time.sleep(1)

# This command reports the actual ambient or slot temperatures of the three sensors.
# It can be chosen whether the evaluated temperature or the sensor value should be reported.
# The temperatures are reported in 1/10 °C: 345 = 34,5 °C

0 {1: '184', 2: '186', 3: '187'}
1 {1: '184', 2: '186', 3: '186'}
2 {1: '184', 2: '186', 3: '186'}
3 {1: '184', 2: '186', 3: '186'}
4 {1: '184', 2: '186', 3: '186'}


In [ ]:
    async def get_temperature(self, sensor: Literal["all", "main", "dif", "boos"]) -> float:
        """Get the current temperature of the temperature controller in Celsius."""

        sensor_mapping = {
            "all": [1, 2, 3],
            "main": [1],
            "dif": [2],
            "boos": [3]
        }

        resp_summary = []
        
        for idx in sensor_mapping[sensor]:
            
            temp_meas = await incubator.send_command(f"RAT{x}", read_timeout=60)

            resp_summary.append(int(temp_meas)/10)      
        
        raise resp_summary/len(resp_summary)

In [ ]:
await incubator.send_command("RTT", read_timeout=60)


In [ ]:
await incubator.send_command("STT700", read_timeout=60)


### Shaking Control

TODO

In [4]:
import serial.tools.list_ports

def list_serial_devices():
    ports = serial.tools.list_ports.comports()
    for port in ports:
        print(f"Device: {port.device}")
        print(f"  Description: {port.description}")
        print(f"  HWID: {port.hwid}")
        print("-" * 40)

if __name__ == "__main__":
    print("Scanning USB serial devices...")
    list_serial_devices()


Scanning USB serial devices...
Device: /dev/cu.debug-console
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.BoseQC35II
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.Bluetooth-Incoming-Port
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.usbserial-140
  Description: USB <-> Serial
  HWID: USB VID:PID=0403:6001 LOCATION=0-1.4
----------------------------------------


### Closing Connection

In [5]:
await incubator.stop()